In [1]:
import pandas as pd
import numpy as np

In [3]:
data2_2 = pd.read_csv("files\\data\\2-2_long_term_frgn.csv", encoding='cp949')
data2_2

,base_date,time,nationality,city,emd_cd,emd_nm,resd_pop_cnt,work_pop_cnt,visit_pop_cnt,resd
0,2018-01-01,1,CAN,제주시,50110250,한림읍,6.3381,0.0000,4.5802,제주
1,2018-01-01,1,CHN,제주시,50110250,한림읍,442.1091,0.0000,5.7051,제주
2,2018-01-01,1,CHN,제주시,50110250,한림읍,0.0000,7.8695,51.3461,그외
3,2018-01-01,1,DEU,제주시,50110250,한림읍,0.0000,0.0000,8.2099,그외
4,2018-01-01,1,ETC,제주시,50110250,한림읍,15.9184,0.0000,2.8758,제주
...,...,...,...,...,...,...,...,...,...,...
23837312,2021-06-30,24,THA,서귀포시,50130620,예래동,0.0000,0.0000,3.3558,제주
23837313,2021-06-30,24,TWN,서귀포시,50130620,예래동,0.0000,0.0000,12.2655,그외
23837314,2021-06-30,24,USA,서귀포시,50130620,예래동,22.0624,0.0000,0.0000,제주
23837315,2021-06-30,24,USA,서귀포시,50130620,예래동,0.0000,0.0000,26.4749,그외


In [4]:
# 필요없는 열 삭제 (날짜, 행정동코드, 행정동명, 배출량만 남김)
data2_2_pre = data2_2.drop(['time', 'nationality', 'city', 'resd'], 1)
data2_2_pre

,base_date,emd_cd,emd_nm,resd_pop_cnt,work_pop_cnt,visit_pop_cnt
0,2018-01-01,50110250,한림읍,6.3381,0.0000,4.5802
1,2018-01-01,50110250,한림읍,442.1091,0.0000,5.7051
2,2018-01-01,50110250,한림읍,0.0000,7.8695,51.3461
3,2018-01-01,50110250,한림읍,0.0000,0.0000,8.2099
4,2018-01-01,50110250,한림읍,15.9184,0.0000,2.8758
...,...,...,...,...,...,...
23837312,2021-06-30,50130620,예래동,0.0000,0.0000,3.3558
23837313,2021-06-30,50130620,예래동,0.0000,0.0000,12.2655
23837314,2021-06-30,50130620,예래동,22.0624,0.0000,0.0000
23837315,2021-06-30,50130620,예래동,0.0000,0.0000,26.4749


In [5]:
# 각 열을 하나의 type으로 통일

# 날짜데이터 datetime으로 변환
data2_2_pre['base_date'] = pd.to_datetime(data2_2_pre['base_date'], format="%Y-%m-%d")

# "알수없음" 삭제
idx = data2_2_pre[data2_2_pre['emd_cd'] == "알수없음"].index
data2_2_pre = data2_2_pre.drop(idx)

# 행정동코드 int형으로 변환
data2_2_pre['emd_cd'] = pd.to_numeric(data2_2_pre['emd_cd'])
# 인구수 int형으로 변환
data2_2_pre['resd_pop_cnt'] = pd.to_numeric(data2_2_pre['resd_pop_cnt'])
data2_2_pre['work_pop_cnt'] = pd.to_numeric(data2_2_pre['work_pop_cnt'])
data2_2_pre['visit_pop_cnt'] = pd.to_numeric(data2_2_pre['visit_pop_cnt'])

data2_2_pre

,base_date,emd_cd,emd_nm,resd_pop_cnt,work_pop_cnt,visit_pop_cnt
0,2018-01-01,50110250,한림읍,6.3381,0.0000,4.5802
1,2018-01-01,50110250,한림읍,442.1091,0.0000,5.7051
2,2018-01-01,50110250,한림읍,0.0000,7.8695,51.3461
3,2018-01-01,50110250,한림읍,0.0000,0.0000,8.2099
4,2018-01-01,50110250,한림읍,15.9184,0.0000,2.8758
...,...,...,...,...,...,...
23837312,2021-06-30,50130620,예래동,0.0000,0.0000,3.3558
23837313,2021-06-30,50130620,예래동,0.0000,0.0000,12.2655
23837314,2021-06-30,50130620,예래동,22.0624,0.0000,0.0000
23837315,2021-06-30,50130620,예래동,0.0000,0.0000,26.4749


In [6]:
# 행정동코드 매개변수로 받아서 해당 구역의 배출량을 월별로 정리
def monthly_output(emdcd):
    emd_data = data2_2_pre[data2_2_pre['emd_cd'] == emdcd]    # 해당 행정동 데이터만 추출
    if len(emd_data != 0):
        emd_data_monthly = emd_data.resample('M', on='base_date').sum()    # 월별로 합산
        emd_data_monthly['emd_cd'] = emdcd
        return emd_data_monthly
    else:
        return

In [7]:
emd_cd_ary = [50110250, 50110253, 50110256, 50110259, 50110310, 50110320, 50110330, 50110510, 50110520, 50110530, 50110540, 50110550, 50110560, 50110570, 50110580, 50110590, 50110600, 50110610, 50110620, 50110630, 50110640, 50110650, 50110660, 50110670, 50110680, 50110690, 50130250, 50130253, 50130259, 50130310, 50130320, 50130510, 50130520, 50130530, 50130540, 50130550, 50130560, 50130570, 50130580, 50130590, 50130600, 50130610, 50130620]

data2_2_result = pd.DataFrame(index=range(0,0), columns=['base_date', 'emd_cd', 'resd_pop_cnt', 'work_pop_cnt', 'visit_pop_cnt'])

for i in emd_cd_ary:
    tmp = monthly_output(i)
    data2_2_result = data2_2_result.append(tmp)

data2_2_result['base_date'] = data2_2_result.index
data2_2_result

,base_date,emd_cd,resd_pop_cnt,work_pop_cnt,visit_pop_cnt
2018-01-31,2018-01-31,50110250,1.019370e+06,112924.6515,315061.6848
2018-02-28,2018-02-28,50110250,9.368096e+05,106187.4097,321053.4022
2018-03-31,2018-03-31,50110250,9.977889e+05,107030.7232,406607.5399
2018-04-30,2018-04-30,50110250,9.078420e+05,110573.1469,364028.2041
2018-05-31,2018-05-31,50110250,8.237094e+05,88514.9394,306747.6089
...,...,...,...,...,...
2021-02-28,2021-02-28,50130620,1.310902e+05,21260.3645,88065.1221
2021-03-31,2021-03-31,50130620,1.351959e+05,24214.5837,94044.3054
2021-04-30,2021-04-30,50130620,1.370498e+05,24437.3744,104790.3736
2021-05-31,2021-05-31,50130620,1.485746e+05,30121.6614,114091.6036


In [8]:
data2_2_result.to_csv("files\\data_preprocessing\\2_2_long_term_frgn_preprocessing_v2.csv", index_label=False)

In [9]:
pd.read_csv("files\\data_preprocessing\\2_2_long_term_frgn_preprocessing_v2.csv")

,base_date,emd_cd,resd_pop_cnt,work_pop_cnt,visit_pop_cnt
2018-01-31,2018-01-31,50110250,1.019370e+06,112924.6515,315061.6848
2018-02-28,2018-02-28,50110250,9.368096e+05,106187.4097,321053.4022
2018-03-31,2018-03-31,50110250,9.977889e+05,107030.7232,406607.5399
2018-04-30,2018-04-30,50110250,9.078420e+05,110573.1469,364028.2041
2018-05-31,2018-05-31,50110250,8.237094e+05,88514.9394,306747.6089
...,...,...,...,...,...
2021-02-28,2021-02-28,50130620,1.310902e+05,21260.3645,88065.1221
2021-03-31,2021-03-31,50130620,1.351959e+05,24214.5837,94044.3054
2021-04-30,2021-04-30,50130620,1.370498e+05,24437.3744,104790.3736
2021-05-31,2021-05-31,50130620,1.485746e+05,30121.6614,114091.6036


In [10]:
# 총인구수 = 거주+근무+방문
data2_2_result['pop_cnt'] = data2_2_result.iloc[:, 3:6].sum(axis=1)
data2_2_result = data2_2_result.drop(['resd_pop_cnt', 'work_pop_cnt', 'visit_pop_cnt'], 1)
data2_2_result

,base_date,emd_cd,pop_cnt
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480
2021-05-31,2021-05-31,50130620,144213.2650


In [11]:
data2_2_result.to_csv("files\\data_preprocessing\\2_2_long_term_frgn_preprocessing.csv", index_label=False)

In [12]:
pd.read_csv("files\\data_preprocessing\\2_2_long_term_frgn_preprocessing.csv")

,base_date,emd_cd,pop_cnt
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480
2021-05-31,2021-05-31,50130620,144213.2650
